<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
z = zipfile.ZipFile("drive/My Drive/additional/train.zip")
z.extractall("transfer")

In [ ]:
import glob
import pandas as pd
dogs = glob.glob("transfer/train/dog.*")
cats = glob.glob("transfer/train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "ans":[0] * len(dogs) + [1] * len(cats)
})
df